In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    ! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
    import setup_google_colab
    setup_google_colab.setup_week1() 
    
import sys
sys.path.append("..")
from common.download_utils import download_week1_resources

download_week1_resources()

In [ ]:
from grader import Grader

In [ ]:
grader = Grader()


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
from ast import literal_eval
import pandas as pd
import numpy as np
from scipy import sparse as sp_sparse

In [ ]:
def read_data(file):
  data = pd.read_csv(file, sep = '\t')
  data['tags'] = data['tags'].apply(literal_eval)
  return data

In [ ]:
train = read_data('data/train.tsv')
validation = read_data('data/validation.tsv')
test = pd.read_csv('data/test.tsv', sep='\t')
train.head()

In [ ]:
X_train, Y_train = train['title'].values, train['tags'].values
X_val, Y_val = validation['title'].values, validation['tags'].values
X_test = test['title'].values
print(X_train)

In [ ]:
import re

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
#print(REPLACE_BY_SPACE_RE)
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
Stopwords = set(stopwords.words('english'))
STOPWORDS = dict()
for i in Stopwords:
  STOPWORDS[i] = 1
for i in Stopwords:
  print(i)

In [ ]:
def text_prepare(text):
  text = text.lower()
  text = REPLACE_BY_SPACE_RE.sub(" ", text)
  text = BAD_SYMBOLS_RE.sub("", text)
  text = text.split()
  #print(text)
  arr = []
  for i in text:
    if STOPWORDS.get(i) == None:
      arr.append(i)
  text = " ".join(arr)
  return text

In [ ]:
def test_text_prepare():
    examples = ["SQL Server - any equivalent of Excel's CHOOSE function?",
                "How to free c++ memory vector<int> * arr?"]
    answers = ["sql server equivalent excels choose function", 
               "free c++ memory vectorint arr"]
    for ex, ans in zip(examples, answers):
      # print("ankit")
       # print(text_prepare(ex))
        if text_prepare(ex) != ans:
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

In [ ]:
print(test_text_prepare())


In [ ]:
prepared_questions = []
for line in open('data/text_prepare_tests.tsv', encoding='utf-8'):
    line = text_prepare(line.strip())
    prepared_questions.append(line)
text_prepare_results = '\n'.join(prepared_questions)

grader.submit_tag('TextPrepare', text_prepare_results)

In [ ]:
X_train = [text_prepare(x) for x in X_train]
X_val = [text_prepare(x) for x in X_val]
X_test = [text_prepare(x) for x in X_test]

In [ ]:
X_train[:7]

In [ ]:
print(Y_train)

In [ ]:
tags_counts = {}
words_counts = {}
for i in Y_train:
  for x in i:
    #print(x)
    if tags_counts.get(x) == None:
      tags_counts[x] = 1
    else:
      tags_counts[x] += 1

for i in X_train:
  for x in i.split():
    if words_counts.get(x) == None:
      words_counts[x] = 1
    else:
      words_counts[x] += 1

print(tags_counts)
print(words_counts)

In [ ]:

most_common_tags = sorted(tags_counts.items(), key=lambda x: x[1], reverse=True)[:3]
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:3]

grader.submit_tag('WordsTagsCount', '%s\n%s' % (','.join(tag for tag, _ in most_common_tags), 
                                                ','.join(word for word, _ in most_common_words)))
print(most_common_tags)
print(most_common_words)

In [ ]:
DICT_SIZE = 5000
SORTED_WORDS = sorted(words_counts.items(), key = lambda x: x[1], reverse = True)[0:5000]
print(SORTED_WORDS)

In [ ]:
WORDS_TO_INDEX = {}
count = 0
for i in SORTED_WORDS:
  WORDS_TO_INDEX[i[0]] = count
  count += 1
print(WORDS_TO_INDEX)
INDEX_TO_WORDS = {}
for (i, j) in WORDS_TO_INDEX.items():
  INDEX_TO_WORDS[j] = i
print(INDEX_TO_WORDS)
ALL_DIFF_WORDS = WORDS_TO_INDEX.keys()
print(ALL_DIFF_WORDS)

In [ ]:
def my_bag_of_words(text, words_to_index, dict_size):
  ans = np.zeros(dict_size, dtype = int)
  #print(ans)
  text = text.split()
  #s = text_prepare(text).split()
  for i in text:
    if words_to_index.get(i) != None:
      ans[words_to_index[i]] += 1
  #print(ans)
  return ans
  

In [ ]:
def test_my_bag_of_words():
    words_to_index = {'hi': 0, 'you': 1, 'me': 2, 'are': 3}
    examples = ['hi how are you']
    answers = [[1, 1, 0, 1]]
    for ex, ans in zip(examples, answers):
        if (my_bag_of_words(ex, words_to_index, 4) != ans).any():
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

In [ ]:
test_my_bag_of_words()

In [ ]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)


In [ ]:
# m = X_train_mybag.toarray()
# flag = 0
# for i in m:
#   for x in i:
#     if x == 1:
#       flag = 1
# print(flag)
# print(m)
# print(X_val_mybag)
# print(X_test_mybag)

In [ ]:
row = X_train_mybag[10].toarray()[0]
non_zero_elements_count = 0
for i in row:
  if i != 0:
    non_zero_elements_count += 1
print(non_zero_elements_count)
grader.submit_tag('BagOfWords', str(non_zero_elements_count))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def tfidf_features(X_train, X_val, X_test):
  tfidf_vectorizer = TfidfVectorizer(token_pattern = r'\S+', ngram_range=(1,2), max_df=0.9, min_df=5,)
  X = tfidf_vectorizer.fit(X_train)
  print(tfidf_vectorizer.get_feature_names())
  #X = tfidf_vectorizer.fit(X_train)
  X_train = tfidf_vectorizer.transform(X_train)
  X_val = tfidf_vectorizer.transform(X_val)
  X_test = tfidf_vectorizer.transform(X_test)
  return X_train, X_val, X_test, tfidf_vectorizer.vocabulary_

In [ ]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}
print(tfidf_reversed_vocab)
count = 0
for (i, x) in tfidf_reversed_vocab.items():
  if x == "c#":
    count += 1
  if x == "c++":
    count += 1
print(count)
#print(X_train_tfidf.toarray())
print(tfidf_vocab["c++"])
print(tfidf_vocab["c#"])

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer


In [ ]:
mlb = MultiLabelBinarizer(classes = sorted(tags_counts.keys()))
print(sorted(tags_counts.keys()))
Y_train = mlb.fit_transform(Y_train)
Y_val = mlb.fit_transform(Y_val)
print(Y_train)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
def train_classifier(X_train, Y_train):
  class_clf = OneVsRestClassifier(LogisticRegression(random_state = 0, max_iter = 10000))
  clf = class_clf.fit(X_train, Y_train)
  return clf

In [ ]:
classifier_mybag = train_classifier(X_train_mybag, Y_train)
classifier_tfidf = train_classifier(X_train_tfidf, Y_train)


In [ ]:
Y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
Y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)
Y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
Y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

In [ ]:
print(Y_val_predicted_labels_mybag)
print(Y_val_predicted_labels_tfidf)

In [ ]:
Y_val_pred_label_inverse = mlb.inverse_transform(Y_val_predicted_labels_tfidf)
Y_val_true = mlb.inverse_transform(Y_val)
print(Y_val_pred_label_inverse)
print(Y_val_true)

In [ ]:
for i in range(60):
  if len(Y_val_pred_label_inverse[i]) == 0:
    Y_val_pred_label_inverse[i] = "TEXT was nothing related to any language or related to many languages"
  print("TEXT : " + str(X_val[i]))
  print("PREDICTED TAGS BY MODEL : " + str(Y_val_pred_label_inverse[i]))
  print("TAG IT REALY HAD ON STACKOVERFLOW : " + str(Y_val_true[i]))
  print("\n")